In [ ]:
%load_ext watermark
%watermark  -d -u -a 'Andreas Mueller, Kyle Kastner, Sebastian Raschka' -v -p numpy,scipy,matplotlib

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

训练数据和测试数据
=====================================

为了评价监督模型的泛化程度，可以将数据分成训练集和测试集:

<img src="figures/train_test_split_matrix.svg" width="60%">

In [ ]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier

iris = load_iris()
X, y = iris.data, iris.target

classifier = KNeighborsClassifier()

考虑到机器学习通常是如何应用的，训练/测试分离的想法是有意义的。现实世界的系统，是根据已知的数据进行训练的，当其他数据(来自客户、传感器或其他来源)到来时，经过训练的分类器必须根据全新的数据进行预测。我们可以在训练期间用训练/测试的分割来模拟这种情况——测试数据是对“未来数据”的模拟，这些数据将在产品阶段进入系统。

特别的，对于iris数据集，150个标签被排序过的，意味着如果我们按比例分割数据，会导致类别分布的大变化。例如，如果都用2/3训练数据和1/3测试数据的比例进行分割，那么训练数据集将只包含类标签0和1的样本，测试集将只包含类标签为2的样本。

在假设所有样本彼此独立的情况下(相对时间序列数据)，如上所述，我们得**在分割数据集之前随机打乱数据集**。

In [ ]:
y

将数据分为训练集和测试集。幸运的是，这是机器学习中的一种常见模式，scikit-learn有一个内建函数，可以将数据分成训练集和测试集。这里，我们用50%的数据做训练，50%做测试。另一种常见比例是80%和20%，其实并没有什么硬性规定。最重要的，是用你的系统在训练中*没见过*的数据来公正的评价你的系统！

In [ ]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, 
                                                    train_size=0.5, 
                                                    random_state=123)
print("Labels for training and testing data")
print(train_y)
print(test_y)

---

**技巧: 分层分割(Stratified Split)**

特别是对于相对较小的数据集，最好对分割进行分层。分层意味着我们在测试和训练集中保持数据集的原始类别比例。例如，在我们如前面的代码示例所示，随机分割数据集之后，我们得到以下以百分比表示的类比例:

In [ ]:
print('All:', np.bincount(y) / float(len(y)) * 100.0)
print('Training:', np.bincount(train_y) / float(len(train_y)) * 100.0)
print('Test:', np.bincount(test_y) / float(len(test_y)) * 100.0)

因此，为了对分割进行分层，我们可以将标签数组作为额外项传给`train_test_split`函数:

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                                                    train_size=0.5, 
                                                    random_state=123,
                                                    stratify=y)

print('All:', np.bincount(y) / float(len(y)) * 100.0)
print('Training:', np.bincount(train_y) / float(len(train_y)) * 100.0)
print('Test:', np.bincount(test_y) / float(len(test_y)) * 100.0)

---

用训练过程中看到过的数据评价分类器的性能，可以会对模型的预测能力抱有错误的信心。在最坏的情况下，模型可能只是简单地记住训练样本，但完全不能对新的、相似的样本进行分类——我们真的不想把这样的系统投入应用！

与其用相同的数据集进行训练和测试，不如用训练/测试分割来评价训练模型在新数据上的表现。

In [ ]:
classifier.fit(train_X, train_y)
pred_y = classifier.predict(test_X)

print("Fraction Correct [Accuracy]:")
print(np.sum(pred_y == test_y) / float(len(test_y)))

也可以对正确和失败的预测进行可视化：

In [ ]:
print('Samples correctly classified:')
correct_idx = np.where(pred_y == test_y)[0]
print(correct_idx)

print('\nSamples incorrectly classified:')
incorrect_idx = np.where(pred_y != test_y)[0]
print(incorrect_idx)

In [ ]:
# Plot two dimensions

colors = ["darkblue", "darkgreen", "gray"]

for n, color in enumerate(colors):
    idx = np.where(test_y == n)[0]
    plt.scatter(test_X[idx, 1], test_X[idx, 2], color=color, label="Class %s" % str(n))

plt.scatter(test_X[incorrect_idx, 1], test_X[incorrect_idx, 2], color="darkred")

plt.xlabel('sepal width [cm]')
plt.ylabel('petal length [cm]')
plt.legend(loc=3)
plt.title("Iris Classification results")
plt.show()

可以看到，错误发生在绿色(类别1)和灰色(类别2)重叠区域。这给了我们关于该添加什么样的特征的启发——任何有助于将类1和类2分开的特征应该都有助于提高分类器性能。

# 练习

打印3个错误预测的真实标签，修改上面使用的散点图代码，以便在2D散点图中用不同的标记可视化和区分这三个样本。你能解释为什么我们的分类器做出了这些错误的预测吗？

In [ ]:
# %load solutions/04_wrong-predictions.py